In [1]:
import rds_config
from sqlalchemy import create_engine

#rds settings
db_host  = rds_config.db_host
db_username = rds_config.db_username
db_password = rds_config.db_password
db_name = rds_config.db_name

#한글 깨짐 현상 해결
#http://yongho1037.tistory.com/570
SQLALCHEMY_DATABASE_URI = 'mysql+pymysql://' + db_username + ':' + db_password + '@' + db_host + '/' + db_name + '?charset=utf8'

engine = create_engine(SQLALCHEMY_DATABASE_URI)
connection = engine.connect()

In [2]:
import pandas as pd
sql = "SELECT media, author, title, content, ml_section FROM newscrawl WHERE media='jtbc' and trim(content) != '' and ml_section > 0"
df = pd.read_sql(sql, connection)
#pd.read_csv("dcdata10.csv")

In [3]:
len(df)

20869

In [4]:
df['content']=df['content'].str.replace(r'\t|\n|\/\/[a-zA-Z가-힣 \n]+|{}|_flash_removeCallback\(\)|[a-zA-Z0-9]+@yonhapnews.co.kr','')

In [5]:
docs_ko=df['content'].str.split('.')
len(docs_ko)

20869

In [6]:
%%time
from konlpy.tag import Mecab; m = Mecab()
pos = lambda d: ['/'.join(p) for p in m.pos(d)]

#[item for sublist in l for item in sublist]  
sentences=[pos(sentence) for doc in docs_ko for sentence in doc]    
#sentences = 
#    texts_ko.append([pos(sentence) for sentence in doc ])
#len(texts_ko)
#from konlpy.tag import Twitter; t = Twitter()
#pos = lambda d: ['/'.join(p) for p in t.pos(d)]
#texts_ko = [pos(doc) for doc in docs_ko]

In [7]:
%%time
from gensim import corpora
dictionary_ko = corpora.Dictionary(sentences)
dictionary_ko.save('dict/jtbc_sen_ko.dict') 

/opt/conda/lib/python3.5/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


CPU times: user 36.9 s, sys: 176 ms, total: 37.1 s
Wall time: 37.2 s


In [270]:
dictionary_ko[2909]

'프리미엄/NNG'

In [8]:
%%time
from gensim.models import word2vec
wv_model_ko = word2vec.Word2Vec(sentences, workers=1, size=300, min_count=10)
wv_model_ko.init_sims(replace=True)
wv_model_ko.save('dict/jtbc_sen_word2vec.model')

CPU times: user 9min 1s, sys: 656 ms, total: 9min 1s
Wall time: 9min 1s


In [277]:
wv_model_ko.most_similar(pos('정치'))

[('기득/NNG', 0.6870841979980469),
 ('퍼블리시티/NNG', 0.6361569166183472),
 ('범야/NNG', 0.612083911895752),
 ('전작/NNG', 0.6095831394195557),
 ('저작/NNG', 0.6072759628295898),
 ('충청/NNP', 0.5973634719848633),
 ('중화/NNG', 0.5893766283988953),
 ('심재/NNG', 0.5718307495117188),
 ('진보/NNG', 0.5626833438873291),
 ('영주/NNG', 0.5533193349838257)]